# DATA EXPLORATION AND MODELLING FOR CHOCOLATE BAR DATASET

## Contribution factor


### Team Number: Team005 


Team Member Name  | Contribution Factor 
------------- | -------------
Karthick Senthil  | 1
Siddhesh Ahire   | 1
Tarjnee Gandhi  | 1
Spandana Reddy   | 1
Aakanksha Warankar | 1

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import json
import folium
import pickle
from branca.colormap import linear
from IPython.display import HTML

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#reading the data from csv
data = pd.read_csv("chocolate.csv")

# 1. Data Cleaning



In [ ]:
#Since the column names have special characters in them we are changing the name of the column names for easy access

new_names = {
     data.columns[0]: 'company',
    'Specific Bean Origin\nor Bar Name': 'bar_origin',
    'REF': 'review_update_value',
    'Review\nDate': 'review_pub_date',
    'Cocoa\nPercent': 'cocoa_percentage',
    'Company\nLocation': 'company_location',
    'Rating': 'rating',
    'Bean\nType': 'bean_type',
    'Broad Bean\nOrigin': 'bean_origin'
}
data = data.rename(new_names, axis='columns')

White spaces are present in the data and they are treated using the strip() function.  

The impossible values present in the data frame are replaced with the NAN values in the attributes  ‘bean_type’ and ‘bean_origin’  and later all the NAN values in the data frame are filled with their respective most repeated value from that attribute. For this simple imputer in imported from sklearn libraries  

In [ ]:
#stripping the % from the rating column and converting it to a numeric column
data['cocoa_percentage'] = (data['cocoa_percentage'].str.strip('%'))
data['cocoa_percentage'] = data['cocoa_percentage'].apply(pd.to_numeric)

#verifying the converted data
data.describe()

#Stripping the bean type for spaces
data['bean_type'] = data['bean_type'].str.strip()

#checking for impossible values
data['bean_type'].unique()

#replacing the unique value with nan
data['bean_type'].replace('\xc2\xa0', np.nan,inplace=True)

#repeating the same for bean origin
data['bean_origin'] = data['bean_origin'].str.strip()
data['bean_origin'].replace('\xc2\xa0', np.nan, inplace=True)

#verifying the values
print(data.head())
print(data.tail())
print(data.describe())
print(data.describe(include=['O']))

Since this is being done as a classification model the rating column has to be changed into a categorical value. To do this the pandas cut function is used which bins the values in to 5 categories. This is then given a label from 1 to 5 for the model to processes.  

In [ ]:
#the grouped values are as above
pd.cut(data.rating, 5)

In [ ]:
#binning the values and assigning labels
data['new_rating'] = pd.cut(data.rating, bins = 5, labels = [1,2,3,4,5])
data.new_rating = data.new_rating.astype(str)

To fill the missing values simple imputer function from pandas is used. This function fills nan values with the most frequent values from the column. This function is chosen because the columns are categorical, and we didn’t know how to use the fillna function on string type.  

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
pd.DataFrame(imp.fit_transform(data), columns=data.columns,  index=data.index)

# 2. Problem Statement:
The chocolate dataset has a rating column which is the final column to be predicted by the model. Since this is a numeric column the usual thing to do is go with a regression. But since we need a confusion matrix and accuracy as the output we have decided to go with a classification model. 
For this rating column is is binned into five different categories and converted into a class label in the previous step.

The aim is to build and compare 3 classification models and present the best model based on accuracy that can best predict the rating of chocolate. We will be using the chocolate dataset which contains 1500 observations and 9 attributes.  


# 3. Learning Algorithms 

The three learning algorithms selected for the model development are
1. RandomForestClassifier
2. KNeighborsClassifier
3. MLPClassifier

The hyperparameter chosen for the tuning is the number of trees in the random forest. We have chosen the values as n= 100, 200, 300, 400, 500

#  Label Encoding and Scaling:
 
Encoding of Categorical or Non-Numeric features: As all Categorical Features are non-numeric, encoding has been done using Label Encoder. 
In the above code, we have used the LabelEncoder class from sklearn preprocessing to transform the labels for 'company','bar_origin','company_location','bean_type', 'bean_origin' , where alphabetically each category is assigned numbers starting from 0.This leads us to another challenge when working with the machine learning models. Since all mathematical models deal with numbers and some numbers are greater than others, this can skew the models leading to inaccurate results. 



We Applied Z-score normalisation (i.e., standardisation) to re-scale each feature of the chocolate data such that each of the re-scaled features has zero mean and unit standard deviation by using the sklearn.preprocessing.StandardScaler2 method. We have many independent variables but the scale of each value has a high range, for example :Bar Origin and Company.that so high and that would be a problem for our ML model, so we need to scale all of our variables with StandardScaler : 

In [ ]:
#creating the X and y variables from the data
to_drop = ["rating","new_rating"]
X = data.drop(to_drop, axis=1).copy()
y = data.new_rating

#using the label encoder to convert the categorical values to numeric values
categorical = ['company','bar_origin','company_location','bean_type', 'bean_origin']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X[feature] = le.fit_transform(X[feature])
#verifying the converted data
print(X.head())
print(X.tail())
#scaling the data using the standard scaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
print(X.head())
print(X.tail())

# 4. Data Partioning:

For modeling purpose all the categorical data is then converted into the numerical data type and then split into train and test dataset so that we can train the model on the training dataset and then predict the rating of the chocolate. All the features are stored into X variable and then encoded into numerical values whereas the Y variable has the label stored I.e. Rating attribute. 

The splitting is done with the test size of 0.3 I.e. in the ratio of 70:30. 
The data is divided sequentially into first 30percent of data into test data and the later 70 percent of the data into train dataset.

The training dataset is then scaled using the standard scaler function.  

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print 'x_train:', x_train
print 'x_test:', x_test
print 'y_train:',y_test
print 'y_train:',y_test

# 5. K Fold Cross Validation:

All the three models employed are then compared and evaluated. The method used for comparison is known as K Fold cross validation statistical method. The value of the k is set as 5 with random state 0. The five folds of the dataset is being printed.  

In [ ]:
#defining kfold validation with n=5
kfold = KFold(n_splits = 5,random_state=0)
for train_index, test_index in kfold.split(x_train):
    print 'train_index:', train_index
    print 'test_index:', test_index

In [ ]:
#creating a models array and adding the models in to the array
models = []
models.append(('KNN',KNeighborsClassifier()))
models.append(('RF - n=100', RandomForestClassifier(n_estimators=100)))
models.append(('RF - n=200', RandomForestClassifier(n_estimators=200)))
models.append(('RF - n=300', RandomForestClassifier(n_estimators=300)))
models.append(('RF - n=400', RandomForestClassifier(n_estimators=400)))
models.append(('RF - n=500', RandomForestClassifier(n_estimators=500)))
models.append(('MLP', MLPClassifier()))
results = []
modelnames = []
scoring = 'accuracy'
for modelname, model in models:
    
    model_result = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(model_result)
    modelnames.append(modelname)
    print(">%s: %f (%f)" % (modelname, model_result.mean(), model_result.std()))
    

From the above comparison results we can see that the random forest has the most highest accuracy among the three algorithms. And at the time of writing this the n=300 gave the best output accuracy among the random forest hyperparameters

# 6. Testing the prediction model on test data

In [ ]:
rf = RandomForestClassifier(n_estimators=300)
rf_train = rf.fit(x_train,y_train)
y_pred = rf_train.predict(x_test)
#calculate the confusion matrix
cm = (confusion_matrix(y_test, y_pred))

#calculate the accuracy
acc = (100*accuracy_score(y_test, y_pred))

#calculate the classification report
cr = classification_report(y_test, y_pred)

#print the output
print(cm)
print("Accuracy: %s%%" % acc)
print(cr)

In [ ]:
#adding the predicted data to the test data as new column
y_test = y_test.to_frame()
dataset = pd.DataFrame.from_records(y_pred)
y_test["predicted"] = dataset[0].values
print(y_test.head())

In [ ]:
#exporting model to pickle
f = open('model.pickle','wb')
pickle.dump(rf_train, f)
f.close()

# 7. Analysis

## 7.1 Feature importance
In order to understand the prediction and which features are important, we can get the feature importances from the predicted model. This can be done by taking the feature importances from the trained model. The top 10 important features are listed.  

In [ ]:
#finding the indices that would sort the array 
sorted_indices = np.argsort(rf_train.feature_importances_)

#finding the most important features and associated importance
variables = rf_train.feature_importances_[sorted_indices]
importance_rating = x_train.columns.values[sorted_indices]

importances = pd.DataFrame({'variable':variables, 'importance':importance_rating})
importances.tail(10)

## 7.2 Map out chocolate rating geographically on the map
To map locaion geographically, we are using Folium python library to visualize data and create an interective map in HTML file. Also we are using GeoJson since We can plot them all at once without a loop.
To explore the rating of the bean geographically, We have choosed to group bean origin and map avarage rating using colour by different colors. 
 - How the rating of chocolate varies as per the bean origin location?
 - Map rating geographically.


In [ ]:
# Average rating of bean_origin
bean_origin_ratings = data.groupby("bean_origin").rating.mean()

# load world contries location json
geo_json_data = json.load(open("world-countries.json"))

# Display colorscale based on rating values
colormap = linear.YlGnBu_09.scale(
    bean_origin_ratings.min(),
    bean_origin_ratings.max())


Created a function to color bean origin location by their rating scale and generate map using GeoJson. 



In [ ]:
# function to map rating on color scale
used_names = []
def make_color(feature):
    current_country = feature['properties']['name']
    if current_country in bean_origin_ratings.index.values:
        used_names.append(current_country)
        return colormap(bean_origin_ratings[feature['properties']['name']])
    else:
        return "floralwhite"
    
#  Map bean orgin with ratings
m = folium.Map([0,0], zoom_start=1.5)

folium.GeoJson(
    geo_json_data,
    name='mean ratings of bean origin',
    style_function=lambda feature: {
        'fillColor': make_color(feature),
        'color': 'grey',
        'weight': 1,
        'dashArray': '3, 3',
        'fillOpacity': 0.8,
    }
).add_to(m)

colormap.caption = 'mean ratings of bean origin'
colormap.add_to(m)

folium.LayerControl().add_to(m)
m.save("bean_origin_ratings.html")
HTML('<iframe src=bean_origin_ratings.html width=800 height=450></iframe>')    

Looking to map we can identify
- The beans which has origin from Brazil, Vietnam and Guatemala used in chocolate has higher average rating.
- Majority of the bean origin are from the South America with higher average ratings

## 7.3 Identify correlation between the feature and How much percentage of cocoa is good for chocolate?

It is interesting to find relation between the features as it will help to predict the values of one of these features given the other features. But we can not be sure of it until we have conclusive evidence regarding the same. To identify relation we are using correlation matrix and heatmap plot.

In [ ]:
# Co-relation between the features
print(data.corr())
sns.heatmap(data.corr())

### KEY OBSERVATION
- Review update date and review update score are highly correlated.
- It is interesting to know that cocoa percentage and ratings are negatively correlated

### Exploring relation between the cocoa percentage and rating

In [ ]:
fig, (a,b) = plt.subplots(1,2,figsize=(20,5))

sns.distplot(data.cocoa_percentage, kde=False, ax=a)
a.set_xlabel("% of cocoa", size=12,color='darkred')
a.set_ylabel("frequency", size=12,color='darkred')
a.set_title("Distribution of cocoa percentage");

sns.boxplot(x=data.new_rating, y=data.cocoa_percentage, ax=b);
b.set_xlabel("New Ratings", size=12,color='darkred')
b.set_ylabel("% of cocoa", size=12,color='darkred')
b.set_title("Comparision of rating with cocoa percentage");

# We will use Seaborn to map
sns.lmplot(x='cocoa_percentage',y='new_rating',fit_reg=False,scatter_kws={"color":"darkred","alpha":0.3,"s":100}
           ,data=data) #syntax for scatter plot

plt.xlabel('Percentage of Cocoa',size=12,color='darkred')
plt.ylabel('Expert Rating of the Bar',size=12,color='darkred')
plt.show()

Looking at the plots above, the relation between the rating and cocoa percentage is quite ambiguous. But, we can notice that majority of the chocolate has cocoa percentage between 70 to 80. When comparing percentage of cocoa with rating it is clearly shows that most chocolate bar ratings are between 2 to 4 and few of them are with the 5 ratings. Interesting fact is that cocoa percentage less than 50% and higer than 90% have rating less than 3 i.e very high percentage of cocoa have a tendency to be not delicious. This might be reason of bitter compounds in cocoa. Also cocoa percentage is not a strong factor to predict chocolate rating as correlation and plots does not imply a well define correlations.